In [1]:
import tensorflow as tf
import torch

import pandas as pd
from pathlib import Path
import time

from rouge_score import rouge_scorer
from rouge_score import scoring

from spacy.lang.en import English
from spacy.lang.de import German

In [2]:
if not tf.config.list_physical_devices('GPU'):
    print("Change runtime to \"GPU runtime\" for faster computations")

In [3]:
BATCH_SIZE = 8

SHUFFEL_SIZE = 1024

learning_rate = 3e-5

model_size = "t5-base"

MAX_ARTICLE_LEN = 512

MAX_HIGHLIGHT_LEN = 150

In [4]:
# tokenizer = T5Tokenizer.from_pretrained(model_size)
# model = TFT5ForConditionalGeneration.from_pretrained(model_size)

# task_specific_params = model.config.task_specific_params
# if task_specific_params is not None:
#     model.config.update(task_specific_params.get("summarization", {}))
    
# pad_token_id = tokenizer.pad_token_id

In [5]:
# val_loss = tf.keras.metrics.Mean(name='val_loss')
# val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

# model.summary()

In [6]:
# ckpt_file = "../models/checkpoint_cross_lingual.ckpt"
# model.load_weights(ckpt_file)

## Pipeline Summarize Then Translate (SumTrans)
both with english and german

In [7]:
data_points = []
filname = "../results/t5_cross_lingual_result.txt"
file = open(filname, "r")
for line in file:
    data_points.append(line)

In [8]:
class SummaryData():
    
    def __init__(self):
        self.id = -1
        self.item_id = -1
        self.language_tag = ''
        self.real_data = ''
        self.pred_data = ''  
        self.translated = ''
        self.translated_real = ''

In [9]:
count = 0
id_count = 0
item_id_count = 1
summary_data = SummaryData()
summary_data_list = []
for point in data_points:
    count += 1
    
    if count == 1:
        summary_data.language_tag = point.split(" ")[2]
        summary_data.id = id_count
        summary_data.item_id =  item_id_count
        if (item_id_count % 8) == 0 and summary_data.language_tag != "en_to_en":
            item_id_count -= 7
        else:
#             print(item_id_count, summary_data.language_tag)
            item_id_count += 1
            
        id_count += 1
    elif count == 3:
        summary_data.pred_data = ": ".join(point.split(": ")[1:])
    elif count == 5:
        summary_data.real_data = ": ".join(point.split(": ")[1:])
        summary_data_list.append(summary_data)
        summary_data = SummaryData()
        count = 0
summary_data_list[0].__dict__, summary_data_list[1].__dict__ 

({'id': 0,
  'item_id': 1,
  'language_tag': 'ger_to_ger',
  'real_data': 'Beatrice sah Rennen auf der Terrasse mit dem Kronprinzen des Golfstaates. Mark 13. Urlaub seit November letzten Jahres, und vierte in einem Monat. Prinzessin kündigte ihren Job bei Sony Pictures in New York vor Weihnachten. Trotzdem wird sie als Vollzeit-Arbeiterin auf der Website ihres Vaters beschrieben.\n',
  'pred_data': 'Prinzessin Beatrice wurde beim Großen Preis von Bahrain gesehen. Die 26-Jährige war mit ihrem langjährigen Freund Dave Clark im Golfstaat gesichtet. Beatrice befand sich in der Startaufstellung hinter dem Kronprinzen von Bahrain. Es wird vermutet, dass auch Formel-1-Legende Sir Jackie Stewart und Komiker Rory Bremner im Turm waren.\n',
  'translated': '',
  'translated_real': ''},
 {'id': 1,
  'item_id': 2,
  'language_tag': 'ger_to_ger',
  'real_data': 'Nach Angaben des türkischen Militärs wurden bei der Flucht vier türkische Soldaten verletzt. Der türkische Präsident Recep Tayyip Erdogan 

In [10]:
ger_to_ger_results = list(filter(lambda data: data.language_tag == "ger_to_ger" ,summary_data_list))
ger_to_en_results = list(filter(lambda data: data.language_tag == "ger_to_en" ,summary_data_list))

en_to_en_results = list(filter(lambda data: data.language_tag == "en_to_en" ,summary_data_list))
en_to_ger_results = list(filter(lambda data: data.language_tag == "en_to_ger" ,summary_data_list))

assert len(ger_to_en_results) == len(ger_to_ger_results) == len(en_to_en_results) == len(en_to_ger_results)

ger_to_ger_results[3].__dict__

{'id': 3,
 'item_id': 4,
 'language_tag': 'ger_to_ger',
 'real_data': 'Kim Callaghan, aus Irland, häufte die Pfunde nach dem Kinderkriegen an. Beschränkt auf Kleidungsgröße 28 sorgte sich Kim, 39, dass sie einem Mann ähnelte. Sie trat der Schlankheitskur bei und ließ zehn Kleidergrößen sowie den 10. Platz fallen.\n',
 'pred_data': 'Kim Callaghan, 39, häufte nach der Geburt ihrer beiden Kinder die Pfunde an und erreichte einen besorgniserregenden 20st 3lb. Nach sportlicher Betätigung und dem Tausch stärkehaltiger Mahlzeiten gegen kleinere Portionen auf ein schlankes Maß geschrumpft. Bevor sie ihre Kleiderschrank verschlankte, beschränkte sie sich auf Kleidungsstücke in Größe 28.\n',
 'translated': '',
 'translated_real': ''}

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [12]:
nlp_en = English()
nlp_ger = German()


sentencizer = nlp_en.create_pipe("sentencizer")
nlp_en.add_pipe(sentencizer)
nlp_ger.add_pipe(sentencizer)


doc_en = nlp_en("This is a sentence. This is another sentence.")
doc_ger = nlp_ger("Das ist ein Satz. Das ist ein weiterer Satz.")
list(doc_en.sents), list(doc_ger.sents)

([This is a sentence., This is another sentence.],
 [Das ist ein Satz., Das ist ein weiterer Satz.])

In [13]:
class MyDatasetGerman(torch.utils.data.Dataset):
    def __init__(self, text):
        self.x = text
        
    def __getitem__(self, index):
        return self.x[index].id, list(nlp_ger(self.x[index].pred_data).sents)
    
    def __len__(self):
        return len(self.x)
    
class MyDatasetEnglish(torch.utils.data.Dataset):
    def __init__(self, text):
        self.x = text
        
    def __getitem__(self, index):
        return self.x[index].id, list(nlp_en(self.x[index].pred_data).sents)
    
    def __len__(self):
        return len(self.x)

In [14]:
ger_data = MyDatasetGerman(ger_to_ger_results)
en_data = MyDatasetEnglish(en_to_en_results)
en_data[0]

(24,
 [Princess Beatrice spotted at Bahrain Grand Prix with long-term boyfriend Dave Clark .,
  Onlooker said 26-year-old was walking behind the Crown Prince of Bahrain .,
  Princes regime accused of violently repressing pro-democracy protests .,
  Sir Jackie Stewart and comedian Rory Bremner watched race from the tower .,
  ])

## Use Fairseq Model

In [15]:
# Load an En-De Transformer model trained on WMT'19 data:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


In [16]:
# Access the underlying TransformerModel
assert isinstance(en2de.models[0], torch.nn.Module)

# Translate from En-De
de = en2de.translate('PyTorch Hub is a pre-trained model repository designed to facilitate research reproducibility.')
assert de == 'PyTorch Hub ist ein vorgefertigtes Modell-Repository, das die Reproduzierbarkeit der Forschung erleichtern soll.'

# to gpu
en2de = en2de.to(device)

In [17]:
result_list = []
for i, data in en_data:
    predictions = en2de.translate(data)
    result_list.append({'id':i, 'translated':predictions})
print(result_list[:])

[{'id': 24, 'translated': ['Prinzessin Beatrice beim Großen Preis von Bahrain mit ihrem langjährigen Freund Dave Clark.', 'Beobachter sagten, der 26-Jährige sei hinter dem Kronprinzen von Bahrain hergelaufen.', 'Prinzen-Regime beschuldigt, prodemokratische Proteste gewaltsam zu unterdrücken.', 'Sir Jackie Stewart und Komiker Rory Bremner verfolgten das Rennen vom Tower aus.', 'Es geht um die Frage, wer die Verantwortung trägt.']}, {'id': 25, 'translated': ['Kurdische Separatisten eröffnen aus großer Entfernung das Feuer und nehmen türkische Soldaten ins Visier, berichtet CNN Türk.', 'Der türkische Ministerpräsident Ahmet Davutoglu sagt: "Die angemessene Antwort auf den abscheulichen Anschlag in Agri wird gegeben". Auch der türkische Präsident Recep Tayyip Erdogan verurteilt die Gewalt scharf.', 'Es geht um die Frage, wer die Verantwortung trägt.']}, {'id': 26, 'translated': ['Random Darknet Shopper kaufte eine Reihe von Artikeln zum Verkauf im Deep Web.', 'Es würde bis zu $100 pro Woch

In [18]:
for i in range(len(result_list)):
    assert en_to_en_results[i].id == result_list[i]['id']
    en_to_en_results[i].translated = "".join(result_list[i]['translated'])
    en_to_en_results[i].translated_real = en_to_ger_results[i].real_data
    assert en_to_en_results[i].item_id == en_to_ger_results[i].item_id
    
en_to_en_results[11].__dict__ 

{'id': 59,
 'item_id': 12,
 'language_tag': 'en_to_en',
 'real_data': 'Experts question if packed out planes are putting passengers at risk . U.S consumer advisory group says minimum space must be stipulated . Safety tests conducted on planes with more leg room than airlines offer .\n',
 'pred_data': 'Tests conducted by the FAA use planes with a 31 inch pitch between each row of seats, a standard which on some airlines has decreased . British Airways has a seat pitch of 31 inches, while easyJet has 29 inches, Thomsons short haul seat pitch is 28 inches, and Virgin Atlantics is 30-31 .\n',
 'translated': 'Bei den von der FAA durchgeführten Tests werden Flugzeuge mit einem Abstand von 31 Zoll zwischen jeder Sitzreihe verwendet, ein Standard, der bei einigen Fluggesellschaften abgenommen hat.British Airways hat einen Sitzabstand von 31 Zoll, während easyJet 29 Zoll, Thomsons Kurzstreckensitzabstand 28 Zoll und Virgin Atlantics 30-31 Zoll hat.Es geht um die Frage, wer die Verantwortung trä

In [19]:
file_name = '../results/en_en_results'
file = open(file_name, "a")
for item in en_to_en_results:
    file.write(str(item.__dict__) + "\n")
    

In [20]:
# Load an En-De Transformer model trained on WMT'19 data:
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe').to(device)

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


In [21]:
result_list_de = []
for i, data in ger_data:
    predictions = de2en.translate(data)
    result_list_de.append({'id':i, 'translated':predictions})
print(result_list_de[:])

[{'id': 0, 'translated': ['Princess Beatrice was seen at the Bahrain Grand Prix.', 'The 26-year-old was spotted in the Gulf state with her longtime boyfriend Dave Clark.', 'Beatrice was on the grid behind the Crown Prince of Bahrain.', 'Formula One legend Sir Jackie Stewart and comedian Rory Bremner are also believed to have been in the tower.', "In fact, it's not."]}, {'id': 1, 'translated': ["Five fighters from the Kurdistan Workers' Party were killed and another wounded in clashes with Turkish forces, the country's military said.", 'Kurdish separatists opened fire from a long distance on Turkish soldiers securing an area in Agri.', "In fact, it's not."]}, {'id': 2, 'translated': ['Random Darknet Shopper bought items that are for sale on the Deep Web.', 'Items included a pair of Nike trainers, fake designer jeans, a scan of a Hungarian passport and a set of British fire keys.', 'The anonymity and encryption of the network has attracted large amounts of illegal activity.', "In fact, i

In [22]:
for i in range(len(result_list_de)):
    assert ger_to_ger_results[i].id == result_list_de[i]['id']
    ger_to_ger_results[i].translated = "".join(result_list_de[i]['translated'])
    ger_to_ger_results[i].translated_real = ger_to_en_results[i].real_data
    assert ger_to_ger_results[i].item_id == ger_to_en_results[i].item_id
    
en_to_en_results[11].__dict__ 

{'id': 59,
 'item_id': 12,
 'language_tag': 'en_to_en',
 'real_data': 'Experts question if packed out planes are putting passengers at risk . U.S consumer advisory group says minimum space must be stipulated . Safety tests conducted on planes with more leg room than airlines offer .\n',
 'pred_data': 'Tests conducted by the FAA use planes with a 31 inch pitch between each row of seats, a standard which on some airlines has decreased . British Airways has a seat pitch of 31 inches, while easyJet has 29 inches, Thomsons short haul seat pitch is 28 inches, and Virgin Atlantics is 30-31 .\n',
 'translated': 'Bei den von der FAA durchgeführten Tests werden Flugzeuge mit einem Abstand von 31 Zoll zwischen jeder Sitzreihe verwendet, ein Standard, der bei einigen Fluggesellschaften abgenommen hat.British Airways hat einen Sitzabstand von 31 Zoll, während easyJet 29 Zoll, Thomsons Kurzstreckensitzabstand 28 Zoll und Virgin Atlantics 30-31 Zoll hat.Es geht um die Frage, wer die Verantwortung trä

In [23]:
file_name = '../results/ger_ger_results'
file = open(file_name, "a")
for item in ger_to_ger_results:
    file.write(str(item.__dict__) + "\n")